In [ ]:
TASKID = 23
df = spark.sql(f"SELECT * FROM InFreGen.pic WHERE state = '5' and taskID = {TASKID} ")

StatementMeta(, , , Waiting, , Waiting)

In [ ]:
display(df)

StatementMeta(, , , Waiting, , Waiting)

SynapseWidget(Synapse.DataFrame, ac3962cd-b471-4ac3-b48c-285f7e2a4e75)

In [ ]:
import zipfile
import os

def pack(taskID, image_paths):
    # Define the name of the zip file, using taskID in the name
    zip_filename = f"task_{taskID}_images.zip"
    
    # Create a zip file in write mode
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for file_path in image_paths:
            if os.path.exists(file_path):
                # Add the image to the zip file, using the picID as the file name within the zip
                zipf.write(file_path, arcname=os.path.basename(file_path))
            else:
                print(f"File {file_path} not found.")
    
    print(f"Images packed into {zip_filename} successfully.")
    return zip_filename

import os
import tarfile
import json
from PIL import Image
from PIL.ExifTags import TAGS

def create_sample_structure(sample_index, image_path, title, metadata):
    """
    Creates the required structure for each sample, including .jpg, .txt, and .json files.
    """
    # Create directory for each sample
    sample_dir = str(sample_index)
    os.makedirs(sample_dir, exist_ok=True)
    
    # Save image file
    img = Image.open(image_path)
    img.save(os.path.join(sample_dir, f"{sample_index}.jpg"))
    
    # Save title in .txt file
    with open(os.path.join(sample_dir, f"{sample_index}.txt"), 'w') as txt_file:
        txt_file.write(title)
    
    # Save metadata in .json file
    with open(os.path.join(sample_dir, f"{sample_index}.json"), 'w') as json_file:
        json.dump(metadata, json_file)
    
    return sample_dir

def pack_samples_to_tar(task_id, samples, tar_filename="/lakehouse/default/Files/0000.tar"):
    """
    Packs samples into a .tar file with a maximum of `max_samples` and size `max_size_mb`.
    """
    # Initialize the .tar file
    with tarfile.open(tar_filename, "w") as tar:
        total_size = 0
        for index, (image_path, title, metadata) in enumerate(samples):
            
            # Create structure for each sample
            sample_dir = create_sample_structure(index, image_path, title, metadata)
            
            # Add sample directory to the .tar file
            tar.add(sample_dir, arcname=os.path.basename(sample_dir))
            
            # Calculate the size
            total_size += sum(os.path.getsize(os.path.join(sample_dir, f)) for f in os.listdir(sample_dir))
            
            # Clean up the sample directory after adding to tar
            for f in os.listdir(sample_dir):
                os.remove(os.path.join(sample_dir, f))

            print(sample_dir)
    print(tar_filename)

StatementMeta(, , , Waiting, , Waiting)

In [ ]:
# Collect ready image paths and prepare samples‘
data = df.select("taskID", "picID", "curPicPath", "State", "Resolution", "keywords").toPandas()
ready_samples = []

for index, row in data.iterrows():
    if row["taskID"] != TASKID or row["State"] != 5:
        continue
    
    taskID = row["taskID"]
    picID = row["picID"]
    finalPicPath = row["curPicPath"]
    keywords = row['keywords']
    reslution = row['Resolution']
    
    if finalPicPath:
        # Set title and metadata for each sample
        title = f"Image {picID} of Task {taskID}. The label is {keywords}."
        metadata = {
            "URL": finalPicPath,
            "picID": picID,
            "taskID": taskID,
            "Resolution": reslution,
            "keyWords": keywords,
        }
        
        # Open the image to get its dimensions
        try:
            with Image.open(finalPicPath) as img:
                metadata["original_width"], metadata["original_height"] = img.size
        except Exception as e:
            print(f"Could not open image {finalPicPath}: {e}")
        
        # Add sample to the list
        ready_samples.append((finalPicPath, title, metadata))

# Pack the images into a tar file
pack_samples_to_tar(task_id=TASKID, samples=ready_samples, tar_filename="/lakehouse/default/Files/" + str(taskID).zfill(4) + ".tar")

StatementMeta(, , , Waiting, , Waiting)

0
1
2
3
4
/lakehouse/default/Files/0023.tar
